In [45]:
import torch
import numpy as np
import warnings

def custom_conv_transpose(matrix, in_channels, out_channels, kernel_size,
                          stride=1, padding=0, output_padding=0, dilation=1,
                          bias=True, padding_mode='zeros'):
    # Генерация случайных значений для смещения (bias)
    bias_val = torch.rand(out_channels) if bias else torch.zeros(out_channels)

    # Проверка режима дополнения (padding_mode)
    if padding_mode != 'zeros':
        raise ValueError('only "zeros" padding_mode in ConvTranspose2d')

    # Генерация случайных значений для весов свертки
    weights = torch.rand(in_channels, out_channels, kernel_size, kernel_size) if isinstance(kernel_size, int) else torch.rand(in_channels, out_channels, *kernel_size)

    # Список для хранения результатов транспонированной свертки
    res_tensor = []

    for l in range(out_channels):
        # Инициализация карты признаков нулями
        feature_map = torch.zeros((matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                                  (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1)

        for c in range(in_channels):
            for i in range(0, matrix.shape[1]):
                for j in range(0, matrix.shape[2]):
                    val = matrix[c][i][j]
                    proizv = val * weights[c][l]

                    # Создание тензора с нулевыми значениями для вычисленных произведений
                    zero_tensor = torch.zeros((weights.shape[2] - 1) * dilation + 1,
                                              (weights.shape[3] - 1) * dilation + 1)

                    # Заполнение тензора вычисленными произведениями с учетом диляции
                    for a in range(0, zero_tensor.shape[0], dilation):
                        for b in range(0, zero_tensor.shape[1], dilation):
                            zero_tensor[a][b] = proizv[a // dilation][b // dilation]

                    # Добавление тензора к существующей карте признаков
                    res = zero_tensor + feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                                                    j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1]
                    feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                                j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1] = res

        # Добавление карты признаков с учетом смещения
        res_tensor.append(feature_map + torch.full(feature_map.shape, bias_val[l]))

    # Применение дополнительного дополнения к результатам, если необходимо
    for t in range(len(res_tensor)):
        if output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
            res_tensor[t] = pad_func(res_tensor[t])

        # Обрезка результатов с учетом указанного дополнения
        res_tensor[t] = res_tensor[t][0 + padding:res_tensor[t].shape[0] - padding,
                                      0 + padding:res_tensor[t].shape[1] - padding]

    # Стекирование результатов в один тензор
    return torch.stack(res_tensor), weights, bias_val


In [46]:
def compare_results(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation, bias=True, padding_mode='zeros'):
    # Получаем результаты из вашей пользовательской функции транспонированной свертки
    my_res, kernel, bias_val = custom_conv_transpose(
        tensor,
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )

    # Создаем экземпляр стандартной функции транспонированной свертки в PyTorch
    torch_function = torch.nn.ConvTranspose2d(
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )
    
    # Назначаем веса и смещение (bias), полученные из пользовательской функции, в стандартную функцию
    torch_function.weight.data = kernel
    torch_function.bias.data = bias_val

    # Сравниваем результаты двух функций с округлением
    result_comparison = torch.all(torch.round(my_res) == torch.round(torch_function(tensor)))

    return result_comparison

# Игнорируем предупреждения (warnings), чтобы уменьшить вывод в консоль
warnings.filterwarnings('ignore')


In [47]:
# Проверка с разными тензорами и параметрами
tensor1 = torch.rand(8, 5, 6)
print(f"Check 1: {compare_results(tensor1, in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')}")

tensor2 = torch.rand(3, 28, 28)
print(f"Check 2: {compare_results(tensor2, in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0, output_padding=0, dilation=3, bias=True, padding_mode='zeros')}")

tensor3 = torch.rand(5, 6, 6)
print(f"Check 3: {compare_results(tensor3, in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros')}")



Check 1: True
Check 2: True
Check 3: True


Доп. Задание: реализовать алгоритм работы транспонированной свертки, через алгоритм двумерной свертки, реализованный в первой лабораторной. Необходимо перерассчитать входные параметры. 

In [55]:
import numpy as np
from torch.nn import ConvTranspose2d
from torch import from_numpy
from numpy.testing import assert_array_equal

def assert_equal_own_and_torch(own, torch, input, weight, bias):
    torch.weight.data = from_numpy(weight).float()
    torch.bias.data = from_numpy(bias).float()
    own_result = np.floor(own)
    torch_result = np.floor(torch(from_numpy(input).float()).detach().numpy())
    print("Результат:")
    print(own_result)
    print("Результат с torch:")
    print(torch_result)
    assert_array_equal(own_result, torch_result)

def conv2d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros'):
    """
    Реализация операции свертки (convolution) для двумерных данных.
    """
    out = []

    batch_size, h_in, w_in = input.shape[0], input.shape[2], input.shape[3]
    kernel_size = (kernel_size, kernel_size) if not isinstance(kernel_size, tuple) else kernel_size
    stride = (stride, stride) if not isinstance(stride, tuple) else stride
    dilation = (dilation, dilation) if not isinstance(dilation, tuple) else dilation

    padding = padding if isinstance(padding, tuple) else (padding, padding)

    h_out = int((h_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
    w_out = int((w_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)

    out = np.zeros((batch_size, out_channels, h_out, w_out))
    for b in range(batch_size):
        for c_out in range(out_channels):
            for y_out in range(h_out):
                for x_out in range(w_out):
                    summation = 0
                    for c_in in range(in_channels):
                        for kernel_y in range(kernel_size[0]):
                            for kernel_x in range(kernel_size[1]):
                                y_in = y_out * stride[0] + kernel_y * dilation[0] - padding[0]
                                x_in = x_out * stride[1] + kernel_x * dilation[1] - padding[1]
                                if 0 <= y_in < h_in and 0 <= x_in < w_in:
                                    summation += input[b, c_in, y_in, x_in] * weight[c_out, c_in, kernel_y, kernel_x]
                                elif padding_mode == 'replicate':
                                    y_in = max(0, min(y_in, h_in - 1))
                                    x_in = max(0, min(x_in, w_in - 1))
                                    summation += input[b, c_in, y_in, x_in] * weight[c_out, c_in, kernel_y, kernel_x]
                    out[b, c_out, y_out, x_out] = summation + (bias[c_out] if bias_enabled else 0)

    return np.array(out)

def conv_transpose2d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros'):
    """
    Реализация операции транспонированной свертки (convolution transpose) для двумерных данных.
    """
    batch_size, in_channels, h_in, w_in = input.shape
    _, out_channels, _, _ = weight.shape
    kernel_size = weight.shape[2]

    out_height = (h_in - 1) * stride - 2 * padding + kernel_size
    out_width = (w_in - 1) * stride - 2 * padding + kernel_size

    output = np.zeros((batch_size, out_channels, out_height, out_width))
    weight_flipped = np.flip(np.flip(weight, axis=-1), axis=-2)

    for b in range(batch_size):
        for c_out in range(out_channels):
            for c_in in range(in_channels):
                conv_result = conv2d(input[b:b+1, c_in:c_in+1, :, :], 
                                     weight_flipped[b:b+1, c_out:c_out+1, :, :], 
                                     bias, 1, 1, kernel_size, stride, 1, dilation, bias_enabled, padding_mode)

                output[b, c_out, :, :] += conv_result.reshape(out_height, out_width)[::stride, ::stride]

    return output

# Тест данные
channel_1 = [[1, 5, 5, 6], [7, 11, 2, 11], [4, 8, 5, 3], [1, 2, 3, 4]]
channel_2 = [[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]]
weight_1 = [[1, 1], [1, 1]]
weight_2 = [[1, 1], [1, 1]]
input_data = np.array([[channel_1, channel_2]])
weight_data = np.array([[weight_1, weight_2], [weight_1, weight_2]])
bias = np.array([0, 0])

# Тест
result = conv_transpose2d(input_data, weight_data, bias, in_channels=1, out_channels=1, kernel_size=2)
torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, stride=1)
assert_equal_own_and_torch(result, torch_result, input_data, weight_data, bias)


Результат:
[[[[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]

  [[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]]]
Результат с torch:
[[[[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]

  [[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]]]
